In [ ]:
!git clone https://github.com/IBM/deep-learning-language-model

Cloning into 'deep-learning-language-model'...
remote: Enumerating objects: 395, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 395 (delta 0), reused 0 (delta 0), pack-reused 393
Receiving objects: 100% (395/395), 37.74 MiB | 24.97 MiB/s, done.
Resolving deltas: 100% (185/185), done.


In [ ]:
from __future__ import print_function
import json
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import Adam
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import requests
import pandas as pd

In [ ]:
path = 'deep-learning-language-model/yelp_100_3.txt'
text = open(path).read().lower()
print('corpus length:', len(text))
char_indices = json.loads(open('deep-learning-language-model/char_indices.txt').read())
indices_char = json.loads(open('deep-learning-language-model/indices_char.txt').read())
chars = sorted(char_indices.keys())
print(indices_char)
#chars = sorted(list(set(text)))
print('total chars:', len(chars))
#char_indices = dict((c, i) for i, c in enumerate(chars))
#indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 256
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(1024, return_sequences=True, input_shape=(maxlen, len(chars))))
model.add(LSTM(512, return_sequences=False))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = Adam(lr=0.002)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

model.load_weights("deep-learning-language-model/transfer_weights")

def sample(preds, temperature=.6):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# train the model, output generated text after each iteration
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    x = np.zeros((1, maxlen, len(chars)))
    preds = model.predict(x, verbose=0)[0]
    
    model.fit(X, y, batch_size=128, epochs=1)

    start_index = random.randint(0, len(text) - maxlen - 1)
    #start_index = char_indices["{"]

    for diversity in [0.2, 0.4, 0.6, 0.8]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)
        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            #print(next_index)
            #print (indices_char)
            next_char = indices_char[str(next_index)]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
model.save_weights("transfer_weights")

corpus length: 71250
{'0': '\n', '1': ' ', '2': '!', '3': '"', '4': '#', '5': '$', '6': '%', '7': '&', '8': "'", '9': '(', '10': ')', '11': '*', '12': '+', '13': ',', '14': '-', '15': '.', '16': '/', '17': '0', '18': '1', '19': '2', '20': '3', '21': '4', '22': '5', '23': '6', '24': '7', '25': '8', '26': '9', '27': ':', '28': ';', '29': '=', '30': '?', '31': '[', '32': ']', '33': 'a', '34': 'b', '35': 'c', '36': 'd', '37': 'e', '38': 'f', '39': 'g', '40': 'h', '41': 'i', '42': 'j', '43': 'k', '44': 'l', '45': 'm', '46': 'n', '47': 'o', '48': 'p', '49': 'q', '50': 'r', '51': 's', '52': 't', '53': 'u', '54': 'v', '55': 'w', '56': 'x', '57': 'y', '58': 'z', '59': '{', '60': '}'}
total chars: 61
nb sequences: 23665
Vectorization...
Build model...


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")



--------------------------------------------------
Iteration 1
185/185 [==============================] - 5041s 27s/step - loss: 1.4968

----- diversity: 0.2
----- Generating with seed: "e this place, or at least like it.  we walked out of there terribly disappointed, i doubt we'll go back.}{this is a nothing but pies place...so i was expecting a little bit more oomph...

dispointing small selection of pie by the slices...also you gotta pu"
e this place, or at least like it.  we walked out of there terribly disappointed, i doubt we'll go back.}{this is a nothing but pies place...so i was expecting a little bit more oomph...

dispointing small selection of pie by the slices...also you gotta pup and the service is no the compery to spe is a little shopp of the salad, and i have to compery to seated a need a bit of meal for me some cheeses of the compery chicken was a bit of perfect for me hearthele for delicious are or meal with out the cook torato salad.  i would be back and salad and 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log


r was potato for a biers and difen anout the best it wasn't and shoppen and the corner of that is a girling that that is that the staff with cashes and the borther and she then the best it wasn't that the best pista was got the best past on the menu and she is a great place

----- diversity: 0.4
----- Generating with seed: "m has was way better.  if you go here.... order a burger.  they are big, juicy and super duper good.  it's also super duper messy.  it's a "fork and knife" type of burger. 

their bloody mary is pretty good... a little too tomato-y.... needed more seasonin"
m has was way better.  if you go here.... order a burger.  they are big, juicy and super duper good.  it's also super duper messy.  it's a "fork and knife" type of burger. 

their bloody mary is pretty good... a little too tomato-y.... needed more seasoning that when i saided tort out the change that is a great staff.

the salad and was so my friends and the most of the copped food coffee of pretis souss of peryo